In [1]:
# First approach to Smart Charging Phase 1 using PuLP.
# Modeled as a PuLP optimisation blending problem
# Started 20 Aug 2020
# Author: Sofia Taylor, Flexible Power Systems
# test new laptop

import numpy as np
import pandas as pd
import datetime as dt
from pulp import *
import pickle
import global_variables as gv
import functions as f
import matplotlib.pyplot as plt
import glob
from testdata_proc import prep_data

import random


In [ ]:
# Import journey and price data

prot_journeys = pickle.load(open('Data/prototype_week','rb'))
test_journeys = pickle.load(open('Data/test_week','rb'))
BAU_profile = pickle.load(open('Data/BAU_profile','rb'))
day_journeys = f.get_daily_data(prot_journeys, gv.DAY)
day_profile = f.create_daily_schedule(BAU_profile, gv.DAY)

In [2]:
all_journeys = prep_data(gv.data_path)
all_journeys.head()

,Route_ID,Branch_ID,Start_Time_of_Route,End_Time_of_Route,Energy_Required,Vehicle_ID,date
0,2135563,513,2019-01-01 07:30:00,01-Jan-2019 13:54:59,20.723753,1,2019-01-01
1,2135564,513,2019-01-01 07:30:00,01-Jan-2019 14:09:00,31.309560,2,2019-01-01
2,2135565,513,2019-01-01 07:30:00,01-Jan-2019 15:30:00,31.153340,3,2019-01-01
3,2135566,513,2019-01-01 07:30:00,01-Jan-2019 12:35:00,33.389727,4,2019-01-01
4,2135567,513,2019-01-01 07:44:59,01-Jan-2019 14:01:00,24.474573,5,2019-01-01


In [3]:
gv.VANS['TEST']

[0, 1, 2, 5, 9, 10, 11, 12, 17, 19]

In [4]:
def limit_vehicles_multishift(journeys, category):
    list_days = list(journeys.date.unique())
    journey_list = []
    for day in list_days:
        daily_journeys = journeys[journeys['date']==day]
        select_journeys = daily_journeys[
            daily_journeys['Vehicle_ID'].isin(gv.VANS[category])]
        journey_list.append(select_journeys)
    return pd.concat(journey_list)

In [5]:
all_journeys.shape

(26524, 7)

In [5]:
# Select a set of vans for the whole time period, and another set for testing
limit_vehicles_multishift(all_journeys, 'PROT')

,Route_ID,Branch_ID,Start_Time_of_Route,End_Time_of_Route,Energy_Required,Vehicle_ID,date
2,2135565,513,2019-01-01 07:30:00,01-Jan-2019 15:30:00,31.153340,3,2019-01-01
3,2135566,513,2019-01-01 07:30:00,01-Jan-2019 12:35:00,33.389727,4,2019-01-01
5,2135568,513,2019-01-01 07:44:59,01-Jan-2019 15:28:00,24.143413,6,2019-01-01
6,2135569,513,2019-01-01 07:44:59,01-Jan-2019 15:18:00,21.708560,7,2019-01-01
7,2135570,513,2019-01-01 07:44:59,01-Jan-2019 15:20:00,19.793687,8,2019-01-01
...,...,...,...,...,...,...,...
37215,2695229,513,2019-12-31 05:50:59,31-Dec-2019 13:53:00,23.019267,13,2019-12-31
37216,2695233,513,2019-12-31 05:56:00,31-Dec-2019 13:13:00,23.943767,14,2019-12-31
37217,2695232,513,2019-12-31 06:00:00,31-Dec-2019 12:52:00,26.178493,15,2019-12-31
37218,2695242,513,2019-12-31 06:09:00,31-Dec-2019 14:37:00,39.633093,16,2019-12-31
